<a href="https://colab.research.google.com/github/yoshihiroo/programming-workshop/blob/master/QC4U_2022/qc4uchapter3_cirq_Japanese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QC4U 第3回 Cirq写経翻訳
2022.10.15 更新

東北大学 [大関先生によるQC4U](https://altema.is.tohoku.ac.jp/QC4U/)の解説コードをもとに、理解を深めるためにCirqへの翻訳をやってみているものです。Cirq初心者ですので、正しくないコードの書き方や理解が間違っているところがあるかも知れませんがご容赦ください。説明の本文は敬意をもってほぼ丸々パクらせていただいております。（掲載については大関先生の了解を得ております。ご承諾ありがとうございます。）

[元にした2022.09.22 第3回の解説コード](https://colab.research.google.com/gist/mohzeki222/03914646f0c7fb8bc4826cddbd44ac23/qc4uchapter3.ipynb)

# Cirqのインストール

前回に引き続きGoogleが提供するCirqを利用しましょう。

In [ ]:
pip install cirq

今回は量子コンピュータを利用した機械学習について紹介します。

In [ ]:
import cirq

### 前回までの振り返り

これまでに学んだものは、H,X,Z、そして制御Zゲートでした。
それぞれの特徴は、H=重ね合わせ、Xは反転、Zは傷をつける（|1>だけ符号をマイナスにする）。
そして制御Zゲートは|11>だけ傷をつけるというものでした。
改めて確率振幅に傷をつけるZというのは偉大だな、と思う次第です。
2個の量子ビットにZを作用するというのを改めてみてみましょう。
ここで一気にどんな変化をするのかを見るために、アダマールを先にかけておきます。

In [ ]:
qc = cirq.Circuit()
q = cirq.LineQubit.range(2)

qc.append(cirq.H.on_each(q))
qc.append(cirq.Z(q[0]))

sim = cirq.Simulator()
res = sim.simulate(qc)

どんな回路になっているのかをみたければprint(qc)を実行しましょう。

In [ ]:
print(qc)

In [ ]:
print(res.final_state_vector.round(5))

この結果を見るとわかりますが、|10>と|11>だけ係数がマイナスになっています。
つまり左側（１つ目の量子ビット）が1のもの２つを傷つけていることがわかります。

逆にZゲートをかけておけば、右側（２つ目の量子ビット）が1のもの２つを傷つけることがわかります。
制御Zゲートと対比すると、１つだけ傷をつけるのか、２つ傷をつけるのかの違いがあります。



さて今日新しく学ぶゲートは制御Xゲートです。
まずはどのような作用をするのか調べてみましょう。
制御量子ビットが|0>にあると意味がないので、|1>にしておきます。
そのためには初期化で実施するか、Xをあらかじめかけておくという方法があります。

In [ ]:
qc2 = cirq.Circuit()
q = cirq.LineQubit.range(2)

qc2.append(cirq.X(q[0]))
qc2.append(cirq.CNOT(q[0], q[1]))

どんな回路になっているのかを見たければprint、もしくはSVGCircuitを実行しましょう。

In [ ]:
#print(qc2)
from cirq.contrib.svg import SVGCircuit
SVGCircuit(qc2)

早速シミュレートしてみてどんな結果になるかをみてみましょう。

何度もシミュレーションを実行することになりますから、せっかくなので自作関数でシミュレーションを実行する部分をひとまとめにしておきましょう。

In [ ]:
import numpy as np
def sim_state(qc,disp=True):
  res = sim.simulate(qc)
  if disp == True:
    print(cirq.dirac_notation(np.array(res.final_state_vector)))
  return res

この関数ではdispというオプション変数を持たせています。
何も入れなければケット表示で状態ベクトルを出力します。

In [ ]:
state = sim_state(qc2)

わざわざdisp = Falseと打つと、表示を消すことができます。

In [ ]:
state = sim_state(qc2, disp = False)

さてさて、この結果を見ると|10>が|11>になった様子が見れます。
つまり制御量子ビットが|1>だった場合に反転をします。
|11>を入力すると、|10>になります。

つまり  
|00> -> |00>、  
|01> -> |01>、  
|10> -> |11>、  
|11> -> |10>、  
といった形で、下2つの状態を入れ替えることになります。

これを利用すると２つに傷をつけるZゲートの作用を変えることができます。
試しに制御XゲートでZゲートを挟んでみます。

どのような作用になるのかを一気に見比べるために、アダマール回路を最初に実施します。

In [ ]:
qc3 = cirq.Circuit()
q = cirq.LineQubit.range(2)

qc3.append(cirq.H.on_each(q))

qc3.append(cirq.CNOT(q[0], q[1]))
qc3.append(cirq.Z(q[1]))
qc3.append(cirq.CNOT(q[0], q[1]))

落ち着いて回路の様子を見てみましょう。

In [ ]:
SVGCircuit(qc3)

さてこれはどんな作用をするでしょうか？予想しながら結果を眺めてみましょう。


In [ ]:
state = sim_state(qc3)

|01>又は|10>だけに傷がつけられて、それ以外の|00>と|11>はそのままです。
これは1の数が奇数か偶数かの判定ができるとも言えます。
二つの量子ビットの間の数の関係で、結果が決まるので、相互に関係をしている、相互作用をしていると言います。

これは量子コンピュータの分野でも登場するイジング模型を記述する重要です。
Zゲートは、Z軸周りに180度回転させるものでした。
代わりにZ軸周りに180度までは回転させず、微調整が効く回転ゲートを利用してみましょう。

In [ ]:
theta = 0.5

qc4 = cirq.Circuit()
q = cirq.LineQubit.range(2)

qc4.append(cirq.H.on_each(q))

qc4.append(cirq.CNOT(q[0], q[1]))
qc4.append(cirq.rz(theta).on(q[1]))
qc4.append(cirq.CNOT(q[0], q[1]))

回路の様子を見てみましょう。

In [ ]:
SVGCircuit(qc4)

他にもrx,ry,rzが存在します。rz(角度、量子ビットの指定）で利用することができます。
早速結果を見てみましょう。

In [ ]:
state = sim_state(qc4)

|00>と|11>にかかる係数が同じで、
それとは異なる形で（複素共役ですが）|01>と|10>に同じ係数がかかっています。
やはり1の数が偶数か、奇数かでその作用が変わることは変わりません。
相互作用の程度として角度を大きくしていくと、２つの状態が分離されていきます。
ただし角度は当然$2\pi$周期ですから、繰り返し近づいたり離れたりします。

先ほど名前を登場させましたけれども、量子ビットの|0>を上向き、|1>を下向きのスピンとして、
磁性体（磁石）の模型として知られるイジング模型を表現することができます。
このように量子コンピュータ上で、物質内部で起こっていることを疑似的に表して、その挙動を調べることを量子シミュレーションと言います。

そうした物質内で起こっていることをシミュレーションするとなると、実際に行われる実験結果との比較をする必要が出てきます。
そうした物理量がどのような値になるか、量子力学では、確率的に起こることがきまりました。
それを反映して測定結果を予言するものは期待値ということになります。
そこでシミュレーションの結果として得られた量子状態ベクトルから、期待値を計算する方法を紹介しましょう。

In [ ]:
obs = [cirq.Z(q[0]), cirq.Z(q[1])]
y = sum(sim.simulate_expectation_values(qc, observables=obs))
print(y)

Operatorの中にPauliという記述を介して、ZIとIZとあります。ZIは１つ目の量子ビットのZ方向のスピンの向きを調べる実験に対応しています。IZは２つ目の量子ビットのZ方向のスピンの向きを調べる実験に対応しています。
この実験を何度も行って得られた平均値を予測するのが期待値です。
重ね合わせの状態から始まり、ちょっとだけイジング模型のダイナミクスに従い、スピンの向きが変化しているのですが、わずかな時間だったので、あまり大きな変化はしていないようです。

### 量子シミュレーション

このイジング模型は制御XゲートとZ回転ゲートで表現されます。
それにより、量子ビット間ないしはスピン間の相互作用をシミュレートします。
それに加えて横磁場と呼ばれることがありますが、X回転ゲートを各量子ビットに作用することで、量子アニーリングマシンで起きていることを真似することができます。

回転角度は、そのシミュレーション時間に対応します。
ただし、一気に回転させるとシミュレーションの精度が悪くなります。
そのため少しずつかけていく必要があります。
この少しずつの回転でシミュレーションをさせる方法を鈴木トロッター分解と言います。

それではちょっとの時間の間だけ、横磁場イジング模型で行われていることをシミュレートする量子回路を用意しましょう。

In [ ]:
class Ising_dynamics(cirq.Gate):

  def __init__(self, n, theta_z, theta_x):
    self.n = n
    #theta_xはnumpy array
    #theta_zもnumpy array
    self.theta_z = theta_z
    self.theta_x = theta_x

  def _num_qubits_(self):
    return self.n

  def _decompose_(self, qubits):
    q = qubits
 
    for k in range(self.n):
      yield cirq.rx(self.theta_x[k]).on(q[k])
    
    if self.n > 1:
      for k in range(self.n-1):
        yield cirq.CNOT(q[k],q[k+1])
        yield cirq.rz(self.theta_z[k+1]).on(q[k+1])
        yield cirq.CNOT(q[k],q[k+1])
      yield cirq.CNOT(q[self.n-1],q[0])
      yield cirq.rz(self.theta_z[0]).on(q[0])
      yield cirq.CNOT(q[self.n-1],q[0])

  def _circuit_diagram_info_(self, args):
    return ["UIsing"] * self.num_qubits()

次に量子シミュレーションの結果、量子ビットないしはスピンがどれだけ揃っているのかを調べることにしましょう。
そのための期待値計算を行う関数を用意します。
今後のことを考えて、それぞれの量子ビットでさまざまな回転角度を与えることができるようにしておきました。
実際にこれを利用して横磁場イジング模型の量子シミュレーションを実行してみましょう。
得られた結果から、Z方向のスピンの期待値を測定した場合の結果を見られるようにしておきましょう。

In [ ]:
def mag_exp(qc,n):
  obs = [cirq.Z(q[i]) for i in range(n)]
  y = np.mean(sim.simulate_expectation_values(qc, observables=obs))
  return y

これらを利用して量子シミュレーションを行う量子回路を組んでみましょう。

In [ ]:
n = 5
#step数
Tall = 50
#途中経過を格納するリスト
m_series = []

qc5 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#重ね合わせの状態から始める
#qc5.append(cirq.H.on_each(q))

#短い時間でのシミュレーションをする
dt = 0.1
#横磁場の大きさ
theta_x = 5.0*np.ones(n)*dt
#相互作用の大きさ
theta_z = 3.0*np.ones(n)*dt

UIsing = Ising_dynamics(n, theta_z, theta_x)
for k in range(Tall):
  qc5.append(UIsing.on(*q))
  m = mag_exp(qc5,n)
  m_series.append(m)


さて量子シミュレーションの結果、磁化（スピンの揃い具合：スピンの平均値）はどのように変化しているでしょうか。

In [ ]:
import matplotlib.pyplot as plt
plt.plot(m_series)
plt.show()

結構複雑な振る舞いをしていることがわかりますね。
最初は全て揃った状態から、次第に崩れていき振動しながら揃ったり崩れたりを繰り返しているようです。

#### sympyの利用
ちなみにもう少しだけ紹介するとsympyでは教科書にある表示で、計算を行うことができて便利です。


In [ ]:
from sympy.physics.quantum.qubit import Qubit
q = Qubit("01")
print(q)

このようなケット表示をベクトル表示にすることができます。

In [ ]:
from sympy.physics.quantum.represent import represent
represent(q)

np.array(represent(q))とするとnumpyのarray（行列）として利用できます。

逆にnumpyのarrayで表された行列（ベクトルも）をそのままケットベクトルで表示してくれます。

In [ ]:
from sympy.physics.quantum.qubit import matrix_to_qubit
matrix_to_qubit(represent(q))

### 量子機械学習へ

さて今日の本題。

（教師あり）機械学習では入力xに対して出力yがあり、その間をつなぐ関数f(x)を真似することを目標とします。
例えば猫の画像をxとしたときに、それが猫であるかどうかyを与える関数を作ろうというものです。
例えば猫であることをy=+1として、猫ではないとすることをy=-1とすると、立派な関数として想像することはできます。
しかしどんな関数であるのかはわかりません。
そこで自分の好きな関数を組み合わせて、それらの組み合わせ方を調整することで、うまく整合性のある関数を作ろうとします。
途中では、実際の猫画像を入れて、正しく猫の識別$y=\pm 1$ができているかをチェックします。
うまくできていなければ、その組み合わせに関わる要素を変更していくというわけです。

自分の好きな関数を用意する部分に、今回皆さんと学習している量子回路を利用してみましょう。

まずは量子回路に、猫の画像を入力するために、入力xに応じて、結果が変わるような単純な回路を作ってみましょう。
量子回路ができることは基本的には「回転」ですから、入力の数値を回転角度に変える必要があります。cosやsinなど三角関数を利用しましょう。これらの三角関数は角度が0から360度（弧度法で$0$から$2\pi$）でその値は-1から1まで変わるものです。これを逆に利用すると-1から+1の値を0から360度（弧度法で$0$から$2\pi$）の値に変えてくれます。
それで入力された数値を回転角度に変えます。
そうやって回すだけの回路を用意しましょう。

In [ ]:
class U_in(cirq.Gate):
  def __init__(self, x, n):
    self.x = x
    self.n = n

  def _num_qubits_(self):
    return self.n

  def _decompose_(self, qubits):
    q = qubits
    angle = np.arcsin(self.x)
    yield cirq.rx(angle).on_each(q)

  def _circuit_diagram_info_(self, args):
    return ["U_in"] * self.num_qubits()

これで入力が-1から+1に変わって行った時に、どのように量子状態が生じるのか、その確率が変化していく様子を見てみましょう。

まずは入力xを与えて、それに対して量子回路が生成されて、測定まで行い、0という状態がどれくらい発生するか、その確率を算出するプログラムを作ってみます。

In [ ]:
def QCLinput(x,n):
  qc = cirq.Circuit()
  q = cirq.LineQubit.range(n)

  u_in = U_in(x,n)
  qc.append(u_in.on(*q))
  
  return qc

これを実行すると、入力xに応じて量子回路が作られます。
その量子回路から出力される結果を読み取れば、入力に応じて変形される量子状態から何かの値が出力されるようになります。
出来上がった量子状態から、先ほどの量子シミュレーションで利用した量子ビットの揃い具合、Z方向のスピンの揃い具合を調べてみましょう。

In [ ]:
n = 3
x = 0.1
q = cirq.LineQubit.range(n)
qc = QCLinput(x,n)
y = mag_exp(qc,n)
print(y)

これでxを入力したら出力yが出てくる量子回路を作ることができました。
これを次々に実行して、xを変えた場合にyがどのように変化するのかをみてみましょう。

In [ ]:
import numpy as np

x_series = np.linspace(-1,1,100)
y_series = []

for x in x_series:
  qc = QCLinput(x,n)
  y = mag_exp(qc,n)
  y_series.append(y)

この結果をプロットして眺めてみましょう。
そのためにはpythonのライブラリからmatplotlibを利用します。

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x_series,y_series)
plt.show()

綺麗な半円を作り出すことができました。これは三角関数を利用したことによる結果です。
ここまでであれば、入力xに対して出力yは半円、つまり
\begin{equation}
y = \sqrt{1-x^2}
\end{equation}
という関数を作り出したということになります。

これ以外にも巧みな関数を作り出して、さまざまなデータに備わる入力xと出力yを説明するための準備をする必要があります。

量子回路では、基本的には各量子ビットを回転をさせることができます。
また量子ビット間を結びつけることで、グローバーのアルゴリズムのように係数を操作することができます。
1つの入力xに対して、量子ビットを複数割り当てて複雑な関数を生み出すことができそうです。

まずは複数の量子ビットが与えられた時に、それぞれを適当に回転させる量子回路を用意します。

In [ ]:
class U_rot(cirq.Gate):
  def __init__(self, n, params):
    self.n = n
    self.params = params

  def _num_qubits_(self):
    return self.n

  def _decompose_(self, qubits):
    q = qubits

    for k in range(self.n):
      yield cirq.rx(self.params[k]).on(q[k])
      yield cirq.ry(self.params[self.n+k]).on(q[k])
      yield cirq.rz(self.params[2*self.n+k]).on(q[k])

  def _circuit_diagram_info_(self, args):
    return ["U_rot"] * self.num_qubits()

ここにparamsという形で回転角度を操作できるようにしておきました。

これで回転角度を調整することでさまざまな関数を作れる状態にすることができました。
この角度を調整することで関数の形を変えることができるようにします。

これを付け加えるだけでも、量子状態はもちろん変化します。
どのような変化をもたらしてくれるか、見てみましょう。
出力yを得るためには、複数の量子ビットによる結果を取りまとめる必要があります。
その際に量子シミュレーションで利用した磁化の期待値の計算方法を利用しましょう。

こうしてできた量子回路は確率的に、各量子ビットで-1と1を出力します。
それらを統合してひとつの結果とするわけです。
その際にどの量子ビットを重要視するのかも、ニューラルネットワークの類似性から、重みとしてパラメータを用意して考えることもできます。
これで準備完了です。適当な回転をを各量子ビットにかけることでどのように入力xと出力yの関係が変わるのかをみてみましょう。

In [ ]:
y_series = []
params = np.random.rand(3*n)*2*np.pi

u_rot = U_rot(n,params)
for x in x_series:
  qc = QCLinput(x,n)
  q = cirq.LineQubit.range(n)
  qc.append(u_rot.on(*q))
  y = mag_exp(qc,n)
  y_series.append(y)

ここで出鱈目なパラメータとしてparamsを用意しました。

In [ ]:
plt.plot(x_series,y_series)
plt.show()

次に制御Zゲートで量子ビット間の係数を変える操作を追加してみましょう。

In [ ]:
class U_ent(cirq.Gate):
  def __init__(self, n):
    self.n = n

  def _num_qubits_(self):
    return self.n

  def _decompose_(self, qubits):
    q = qubits
    if self.n > 1:
      for k in range(self.n-1):
        yield cirq.CZ(q[k],q[k+1])
      yield cirq.CZ(q[self.n-1],q[0])

  def _circuit_diagram_info_(self, args):
    return ["U_ent"] * self.num_qubits()

これらを組み合わせるとどのように変化するでしょうか。

In [ ]:
y_series = []

u_rot = U_rot(n,params)
u_ent = U_ent(n)

for x in x_series:
  qc = QCLinput(x,n)
  q = cirq.LineQubit.range(n)
  qc.append(u_ent.on(*q))
  qc.append(u_rot.on(*q))
  y = mag_exp(qc,n)
  y_series.append(y)

さていざ結果はどうでしょうか。

In [ ]:
plt.plot(x_series,y_series)
plt.show()

まるで先ほどまでとは異なる関数形になりました。
制御Zゲートは|11>だけに傷をつけるという作用がありました。
その効果が出ているのでしょうかね。

これを繰り返すだけでも多少は複雑な関数になるかもしれませんね。

In [ ]:
y_series = []

depth = 3
params = np.random.rand(3*depth*n)

u_ent = U_ent(n)

for x in x_series:
  qc = QCLinput(x,n)
  q = cirq.LineQubit.range(n)
  for d in range(depth):
    qc.append(u_ent.on(*q))
    u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
    qc.append(u_rot.on(*q))
  y = mag_exp(qc,n)
  y_series.append(y)

In [ ]:
plt.plot(x_series,y_series)
plt.show()

また形が変わりましたね。

ただこれは出鱈目にパラメータを割り振った結果であり、そして何に合わせるべきかも指定していないものでした。
機械学習では、データが事前に与えられて、それに沿った関数の形を推定することを目標としています。
というわけで、データを用意してみましょう。

In [ ]:
ntrain = 10
func = lambda x: 0.5*x**3
xtrain = 2*np.random.rand(ntrain)-1
ytrain = func(xtrain)

In [ ]:
plt.scatter(xtrain,ytrain)

このような断片的な情報から元になっている関数の形（今の場合はfunc）を推定するということを目指します。

出鱈目なパラメータを動かして、もらったデータに合わせていくということを目指します。
そのためデータと量子回路が弾き出してきた結果が、どれだけ異なるかを明らかにする必要があります。

In [ ]:
def cost_func(params):
    u_ent = U_ent(n)
    cost_total = 0
    for k in range(ntrain):
      qc = QCLinput(xtrain[k],n)
      q = cirq.LineQubit.range(n)
      for d in range(depth):
        qc.append(u_ent.on(*q))
        u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
        qc.append(u_rot.on(*q))
      y = mag_exp(qc,n)

      #データと量子回路の出力結果のズレを計算
      cost = 0.5*(ytrain[k] - y)**2
      cost_total += cost

    #訓練データの個数で割り算することで平均誤差を計算
    cost_total = cost_total/ntrain

    return cost_total

このコスト関数をパラメータを変化させて、できるだけ小さくなるように「最適化」をすることで、
できるだけデータに合わせるというわけです。
やってみましょう。

In [ ]:
#warningが多発する場合に抑制するコマンド
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from scipy.optimize import minimize
result = minimize(cost_func, params, method="COBYLA", options={"maxiter": 100})

どれだけの近づき方をしているのかをみたいときは、result.funです。

In [ ]:
result.fun

結果を取り出すためには、result.xとうちます。

In [ ]:
result.x

これが指定された回数（maxiter）だけ最適化を試みた結果、得られたパラメータのセットです。

この結果を用いてグラフをプロットしてみましょう。

In [ ]:
y_series = []
params = result.x
u_ent = U_ent(n)
for x in x_series:
  qc = QCLinput(x,n)
  q = cirq.LineQubit.range(n)
  for d in range(depth):
    qc.append(u_ent.on(*q))
    u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
    qc.append(u_rot.on(*q))
  y = mag_exp(qc,n)
  y_series.append(y)

データも重ね書きして、元にしている関数もせっかくなので重ねて書いてみましょう。

In [ ]:
y_correct = func(x_series)

In [ ]:
plt.scatter(xtrain,ytrain)
plt.plot(x_series,y_correct)
plt.plot(x_series,y_series)
plt.show()

いかがでしょうか。
うまく合わせられましたか？

このようにしてできた深い方向への量子回路を関数として残しておきましょう。

In [ ]:
def forward(x,n,depth,params):
    qc = QCLinput(x,n)
    q = cirq.LineQubit.range(n)
    u_ent = U_ent(n)
    for d in range(depth):
      qc.append(u_ent.on(*q))
      u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
      qc.append(u_rot.on(*q))
    y = mag_exp(qc,n)
    return y

ちなみに機械学習でデータに合わせるために必要なのは、
いかにして非自明な動きをするか、その非自明な動きを統合するのか、ということに尽きます。
ここまでで制御Zゲートと回転だけを利用しましたので、簡単なものです。
ここで複雑なものにしていくために、試しに先ほどのイジング模型の量子シミュレーションを利用してみましょう。

In [ ]:
#横磁場の大きさ
theta_x = np.random.randn(n)
#相互作用の大きさ
theta_z = np.random.randn(n)

def cost_func2(params):
    UIsing = Ising_dynamics(n, theta_z, theta_x)

    cost_total = 0
    for k in range(ntrain):
      qc = QCLinput(xtrain[k],n)
      q = cirq.LineQubit.range(n)
      for d in range(depth):
        qc.append(UIsing.on(*q))
        u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
        qc.append(u_rot.on(*q))
      y = mag_exp(qc,n)

      #データと量子回路の出力結果のズレを計算
      cost = 0.5*(ytrain[k] - y)**2
      cost_total += cost

    #訓練データの個数で割り算することで平均誤差を計算
    cost_total = cost_total/ntrain

    return cost_total

In [ ]:
params = np.random.rand(3*depth*n)*2*np.pi
result = minimize(cost_func2, params, method="COBYLA", options={"maxiter": 100})

同じようにどれだけ近づくことができたのかを見るにはresult.fun

In [ ]:
result.fun

結果として得られたパラメータを見るにはresult.xですね。

In [ ]:
result.x

もう少し最適化を繰り返したいという場合には、result.xを初期解として続けてみましょう。

In [ ]:
#result = minimize(cost_func2, result.x, method="COBYLA", options={"maxiter": 100})

この結果とデータを比較してみましょう。

In [ ]:
y_series = []
params = result.x

UIsing = Ising_dynamics(n, theta_z, theta_x)

for x in x_series:
  qc = QCLinput(x,n)
  q = cirq.LineQubit.range(n)
  for d in range(depth):
    qc.append(UIsing.on(*q))
    u_rot = U_rot(n,params[d*3*n:(d+1)*3*n])
    qc.append(u_rot.on(*q))
  y = mag_exp(qc,n)
  y_series.append(y)

In [ ]:
plt.scatter(xtrain,ytrain)
plt.plot(x_series,y_correct)
plt.plot(x_series,y_series)
plt.show()

ただここでの方法（COBYLA、他にもNelder-Mead、Powellなどでも最適化可能）は勾配を利用しない最適化方法で適切とは言えません。
やはり深層学習と同じく勾配を利用して、効率的に計算を進める方が確実でしょう。

パラメータを少しずらして微分をすることで勾配を計算するのが機械学習での通常のやり方ですが、
量子回路の場合には、回転させる角度など、利用するパラメータとそのパラメータを持つ量子回路の特徴を利用すると、$\pi/2$ずらしたものと$-\pi/2$ずらしたものを引き算すれば微分の計算と同じ結果を計算することができます。

ただし微分のためには、現在のパラメータにおける量子回路、パラメータをシフトをした量子回路による計算結果を必要とするため、非常に計算量的に重い計算を何度も行う必要があります。

In [ ]:
def calc_grad(n,depth,xtrain,ytrain,params):    
  grad = np.zeros_like(params)
  cost_data = np.zeros(ntrain)
  shifted = params.copy()
    
  for k in range(ntrain):
    x = xtrain[k]
    y = forward(x,n,depth,params)
    cost_data[k] = - (ytrain[k] - y)
    
    for i in range(len(params)):
      shifted[i] += np.pi/2
      y1 = forward(x,n,depth,shifted)    
      shifted[i] -= np.pi
      y2 = forward(x,n,depth,shifted)    
      gradient = 0.5 * (y1-y2)
      grad[i] += cost_data[k]*gradient/ntrain
      shifted[i] += np.pi/2

  return grad, cost_data

In [ ]:
params = np.random.rand(3*depth*n)*2*np.pi

eta = 1.0
Tall = 20
cost_series = []

In [ ]:
for t in range(Tall):
  grad, cost_data = calc_grad(n,depth,xtrain,ytrain,params)
  
  cost = np.sum(cost_data**2)/len(cost_data)
  cost_series.append(cost)
  
  params = params - eta*grad

In [ ]:
plt.plot(cost_series)
plt.show()

In [ ]:
y_series = []

for x in x_series:
  y = forward(x,n,depth,params)
  y_series.append(y)

In [ ]:
plt.scatter(xtrain,ytrain)
plt.plot(x_series,y_correct)
plt.plot(x_series,y_series)
plt.show()